# CPS Labor Force Nonparticipation by Reason

Brian Dew, @bd_econ

Replaces CPS_PrimeLF_Contrib.ipynb

March 19, 2019

Replication using python of a very clever chart by Ernie Tedeschi (@ernietedeschi).

[Definitions](https://www.frbatlanta.org/chcs/human-capital-currents/2015/0612-measuring-labor-market-status-using-basic-data.aspx) of labor market status come from the FRB of Atlanta.

In [1]:
# Import preliminaries
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np

import os

os.chdir('/home/brian/Documents/CPS/data/clean/')

In [2]:
# Age and sex demographics in March 2001
filter_cols = ['AGE', 'FEMALE', 'NILFREASON', 'BASICWGT', 'MONTH']
df = (pd.read_feather('cps2001.ft', columns=filter_cols)
        .query('MONTH == 3 and 18 <= AGE <= 64'))

# Create dictionary with sex and age distribution in March 2001
dem_2001 = {}
for sex in [0, 1]:
    dfs = df[df['FEMALE'] == sex]
    age_dict = (dfs.groupby('AGE').BASICWGT.sum()
                / dfs.BASICWGT.sum()).to_dict()
    for age in age_dict.keys():
        dem_2001[(sex, age)] = age_dict[age]

In [3]:
results = pd.DataFrame()

for year in range(2000, 2020):
    annual_data = (pd.read_feather(f'cps{year}.ft', columns=filter_cols)
                     .query('18 <= AGE <= 64'))
    for month, df in annual_data.groupby('MONTH'):
    
        date = pd.to_datetime(f'{year}-{month}-01')

        dem_adj = {}
        for sex in [0, 1]:
            dfs = df[df['FEMALE'] == sex]
            age_dict = (dfs.groupby('AGE').BASICWGT.sum()
                        / dfs.BASICWGT.sum()).to_dict()
            for age in age_dict.keys():
                dem_adj[(sex, age)] = dem_2001[(sex, age)] / age_dict[age]

        df['COMBO'] = list(zip(df['FEMALE'], df['AGE']))
        df['ADJWGT'] = df['COMBO'].map(dem_adj) * df['BASICWGT']

        s = df.groupby('NILFREASON').ADJWGT.sum() / df.ADJWGT.sum()
        unadj = (df.groupby('NILFREASON').BASICWGT.sum()
                 / df.BASICWGT.sum()).nan
        monthly_result = s.to_frame(name=date).T
        monthly_result.at[date, 'nan'] = s.nan - unadj

        results = results.append(monthly_result)

results = results.rename({'nan': 'Demographics'}, axis=1).rolling(12).mean()
results = results.dropna().loc['2001-03-01':] * 100
final = results.iloc[0] - results

In [4]:
final = results.iloc[0] - results

In [5]:
%matplotlib inline
title='Contribution to age 18-64 labor force non-participation'
final.plot(figsize=(4, 9), title=title);

In [6]:
subset = ['Disabled/Ill', 'Discouraged', 'Family', 'Retired', 'School']

file = '/home/brian/Documents/uschartbook/chartbook/data/nilf.csv'

final[subset].to_csv(file, index_label='date', float_format='%g')

In [7]:
colors = {'Disabled/Ill': 'green!80!blue', 'Discouraged': 'blue', 'Family': 'red', 
          'Retired': 'orange', 'School': 'cyan'}

In [8]:
date = results.index[-1].strftime('%Y-%m-%d')

myfile = open('/home/brian/Documents/uschartbook/chartbook/text/nilf_nodes.txt', 'w')

for name, color in colors.items():
    
    val = final[name].iloc[-1]
    
    valtext = f'{val:.1f}'

    text2 = (f'\\node[label={{0:{{\\footnotesize {valtext}}}}}, circle, {color}, fill, inner sep=1.35pt] at'+
             f'(axis cs:{date}, {val}) {{}};')

    myfile.write(text2)
    
myfile.close()

In [24]:
ltdate = results.index[-1].strftime('%B %Y')
tot = abs(final.iloc[-1].sum())
dem = abs(final['Demographics'].iloc[-1].sum())
sch = abs(final['School'].iloc[-1].sum())
dis = abs(final['Disabled/Ill'].iloc[-1].sum())
ret = abs(final['Retired'].iloc[-1].sum())

In [25]:
text = (f'From March 2001 to the latest available month, {ltdate}, '+
        f'an additional {tot:.1f} percent of the age 18--64 population '+
        f'left the labor force. The larger-than-normal population '+
        f'cohort born after World War II is reaching retirement age in '+
        f'this period. Such demographic effects explain {dem:.1f} '+
        f'percent of the cumulative decrease. Additionally, '+
        f'young people are staying in school longer, on average, '+
        f'reduced the age 18--64 labor force by {dem:.1f} percent. '+
        f'Disability or illness reduced the labor force by '+
        f'another {dis:.1f} percent. Less retirement among those age '+
        f'18--64 increased the labor force by {ret:.1f} percent.')